# Predicting

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings('ignore')

### Loading table

In [2]:
table = pd.read_csv('BR-21.csv')

----------

# Function

In [3]:
already_fetched = {}

In [4]:
def formatstr(string):
    return string.lower().replace(" ","").replace("á","a").replace("ã","a").replace("ê","e").replace("é","e").replace("í","i").replace("ó","o").replace("õ","o").replace("ú","u")

In [5]:
def calc_precision(df, result):
    p = len(df[(df['result'] == result) & (df['pred'] == result)]) / len(df[df['pred'] == result])
    return p

In [6]:
def calc_precision_with_perc(df, result, perc):
    right = len(df[(df['prob' + result] >= perc) & (df['result'] == result )])
    wrong = len(df[(df['prob' + result] >= perc) & (df['result'] != result )])
    if (right + wrong) == 0:
        return [0,0]
    else:
        return [right + wrong, right / (right + wrong)]

In [7]:
def predict(links): 
    
    global already_fetched
    not_fetched_yet = []
    
    df = table.copy()
    
    for link in links:
        if (link in already_fetched):
            df = df.append(already_fetched[link], ignore_index=True)
        else:
            not_fetched_yet.append(link)
            
    if (len(not_fetched_yet) > 0):
        PATH = '/home/matheus/chromedriver'
        driver = webdriver.Chrome(PATH)
    
    for link in not_fetched_yet:
        driver.get(link)
            
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "strong.ficha-jogo__time__jogadores__lista__jogador-identificacao"))
            )
            team_names_long = driver.find_elements_by_css_selector("h1.placar__equipe__header__nome")
            team_names_short = driver.find_elements_by_css_selector("h1.ficha-jogo__time__header-sigla")
        except:
            driver.quit()
            print('ERROR: Could not find strong.ficha-jogo...')

        HOME_LONG = formatstr(team_names_long[0].text)
        AWAY_LONG = formatstr(team_names_long[1].text)
        HOME_SHORT = team_names_short[0].text
        AWAY_SHORT = team_names_short[1].text

        ### Getting Tacticals and Players
        tacticals = driver.find_elements_by_class_name("ficha-jogo__time__esquema")

        home_defenders_number = int(tacticals[0].text.replace(" ","").split("-")[0])
        home_midfielders_number = int(tacticals[0].text.replace(" ","").split("-")[1])
        home_forwards_number = int(tacticals[0].text.replace(" ","").split("-")[2])

        away_defenders_number = int(tacticals[1].text.replace(" ","").split("-")[0])
        away_midfielders_number = int(tacticals[1].text.replace(" ","").split("-")[1])
        away_forwards_number = int(tacticals[1].text.replace(" ","").split("-")[2])

        home_players_container = driver.find_elements_by_class_name("ficha-jogo__time__jogadores__lista")[0]
        away_players_container = driver.find_elements_by_class_name("ficha-jogo__time__jogadores__lista")[1]

        home_players = []
        away_players = []

        lis = home_players_container.find_elements_by_tag_name("li")
        for each in lis:
            if (each.get_attribute("class") == "ficha-jogo__time__jogadores__lista__jogador") | (each.get_attribute("class") == "ficha-jogo__time__jogadores__lista__jogador ficha-jogo__time__jogadores__lista__jogador__substituido ficha-jogo__time__jogadores__lista__jogador__substituido-sai") | (each.get_attribute("class") == "ficha-jogo__time__jogadores__lista__jogador ficha-jogo__time__jogadores__lista__jogador__expulso") | (each.get_attribute("class") == "ficha-jogo__time__jogadores__lista__jogador ficha-jogo__time__jogadores__lista__jogador__substituido ficha-jogo__time__jogadores__lista__jogador__substituido-sai ficha-jogo__time__jogadores__lista__jogador__expulso"):
                name = each.find_element_by_css_selector("strong.ficha-jogo__time__jogadores__lista__jogador-identificacao").text
                home_players.append(name)

        lis = away_players_container.find_elements_by_tag_name("li")
        for each in lis:
            if (each.get_attribute("class") == "ficha-jogo__time__jogadores__lista__jogador") | (each.get_attribute("class") == "ficha-jogo__time__jogadores__lista__jogador ficha-jogo__time__jogadores__lista__jogador__substituido ficha-jogo__time__jogadores__lista__jogador__substituido-sai") | (each.get_attribute("class") == "ficha-jogo__time__jogadores__lista__jogador ficha-jogo__time__jogadores__lista__jogador__expulso") | (each.get_attribute("class") == "ficha-jogo__time__jogadores__lista__jogador ficha-jogo__time__jogadores__lista__jogador__substituido ficha-jogo__time__jogadores__lista__jogador__substituido-sai ficha-jogo__time__jogadores__lista__jogador__expulso"):
                name = each.find_element_by_css_selector("strong.ficha-jogo__time__jogadores__lista__jogador-identificacao").text
                away_players.append(name)

        for i, x in enumerate(home_players):
            home_players[i] = formatstr(x)

        for i, x in enumerate(away_players):
            away_players[i] = formatstr(x)

        home_defenders_to_fill = 5 - home_defenders_number
        home_midfielders_to_fill = 6 - home_midfielders_number
        home_forwards_to_fill = 5 - home_forwards_number

        away_defenders_to_fill = 5 - away_defenders_number
        away_midfielders_to_fill = 6 - away_midfielders_number
        away_forwards_to_fill = 5 - away_forwards_number

        for i in range(home_defenders_to_fill):
            home_players.insert(home_defenders_number+1+i, "none")

        for i in range(home_midfielders_to_fill):
            home_players.insert(home_midfielders_number+6+i, "none")

        for i in range(home_forwards_to_fill):
            home_players.insert(home_forwards_number+12+i, "none")        

        for i in range(away_defenders_to_fill):
            away_players.insert(away_defenders_number+1+i, "none")

        for i in range(away_midfielders_to_fill):
            away_players.insert(away_midfielders_number+6+i, "none")

        for i in range(away_forwards_to_fill):
            away_players.insert(away_forwards_number+12+i, "none")

        for i,x in enumerate(home_players):
            home_players[i] = HOME_SHORT + '_' + x

        for i,x in enumerate(away_players):
            away_players[i] = AWAY_SHORT + '_' + x

        match = [0] + [HOME_LONG.upper()] + [AWAY_LONG.upper()] + home_players + away_players + [0] + [0] + ['D']

        series = pd.Series(match, index=table.columns)
        df = df.append(series, ignore_index=True)
        
        already_fetched[link] = series

        time.sleep(1)
    
    X = df.iloc[:, 1:-3].values
    
    y = df.iloc[:,-1].values
        
    labelencoder = LabelEncoder()

    for i in range(36):
        X[:,i] = labelencoder.fit_transform(X[:,i])
        
    n_links = len(links)
        
    to_predict = X[-n_links:]
    X = X[:-n_links]
    y = y[:-n_links]
    
    model = RandomForestClassifier(criterion='gini',min_samples_leaf=10,min_samples_split=5,n_estimators=70)
    model.fit(X,y)        
    pred = model.predict(to_predict)
    proba = model.predict_proba(to_predict)
    classes = model.classes_

    probs_columns = {}
    
    for i in range(len(classes)):
        lst = []
        for j in range(proba.shape[0]):
            lst.append(proba[j,i])
        probs_columns[classes[i]] = lst     
    
    df_predict = df.iloc[-n_links:, 0:-3]
    df_predict['pred'] = pred
    
    if ('H' in probs_columns):
        df_predict['probH'] = probs_columns['H']
    if ('A' in probs_columns):
        df_predict['probA'] = probs_columns['A']
    if ('D' in probs_columns):
        df_predict['probD'] = probs_columns['D']   
        
    return df_predict[['HTEAM','ATEAM','pred', 'probH','probA','probD']]

In [8]:
def random_forest_matchweek(matchweek):
    predict_df = pd.DataFrame()
    hometeams_list = []
    awayteams_list = []
    real_result_list = []
    predictions_list = []
    probA_list = []
    probD_list = []
    probH_list = []
    
    acc_df = table[table['MATCHWEEK'] <= matchweek]
    matchweek_df = table[table['MATCHWEEK'] == matchweek]
    
    for i in range(10):
        hometeam = matchweek_df.iloc[i,1]
        awayteam = matchweek_df.iloc[i,2]
        
        df = acc_df[(acc_df['HTEAM'] == hometeam) | (acc_df['ATEAM'] == awayteam)]
        X = df.iloc[:, 1:-3].values
    
        y = df.iloc[:,-1].values
        real_result = matchweek_df.iloc[i,-1]
        
        labelencoder = LabelEncoder()

        for i in range(36):
            X[:,i] = labelencoder.fit_transform(X[:,i])
        
        to_predict = X[-1]
        X = X[:-1]
    
        y = y[:-1]
        
        model = RandomForestClassifier(criterion='gini',min_samples_leaf=10,min_samples_split=5,n_estimators=70)
        model.fit(X,y)        
        to_predict = to_predict.reshape(1,-1)
        pred = model.predict(to_predict)
        proba = model.predict_proba(to_predict)
        classes = model.classes_
        
        hometeams_list.append(hometeam)
        awayteams_list.append(awayteam)
        real_result_list.append(real_result)
        predictions_list.append(pred[0])
        
        if 'A' in classes:
            ind = classes.tolist().index('A')
            probA_list.append('{:.1%}'.format(proba[:,ind][0]))
        else:
            probA_list.append(0.0)

        if 'D' in classes:
            ind = classes.tolist().index('D')
            probD_list.append('{:.1%}'.format(proba[:,ind][0]))
        else:
            probD_list.append(0.0)

        if 'H' in classes:
            ind = classes.tolist().index('H')
            probH_list.append('{:.1%}'.format(proba[:,ind][0]))
        else:
            probH_list.append(0.0)
    
    predict_df['Home'] = hometeams_list
    predict_df['Away'] = awayteams_list
    predict_df['result'] = real_result_list
    predict_df['pred'] = predictions_list
    predict_df['probH'] = probH_list
    predict_df['probD'] = probD_list
    predict_df['probA'] = probA_list
    
    acertos = len(predict_df[predict_df['result'] == predict_df['pred']])
    print(acertos/10)
    
    return predict_df

Check out how to use this function below. We can pass any past matchweek and it will create a Random Forest model and train it with data from matchweek 1 to the last matchweek we could get at the time of the matchweek we chose.
<br><br>
- result: real result of the match
- pred: our prediction (Hometeam winning, Awayteam winning or Draw)
- probs: probability of each result

In [9]:
random_forest_matchweek(34)

0.6


,Home,Away,result,pred,probH,probD,probA
0,CORINTHIANS,SANTOS,H,H,45.8%,37.3%,16.9%
1,ATLETICO-MG,JUVENTUDE,H,H,66.0%,23.8%,10.2%
2,BRAGANTINO,SPORT,H,H,48.6%,34.5%,17.0%
3,FORTALEZA,PALMEIRAS,H,A,34.6%,22.1%,43.3%
4,CHAPECOENSE,GREMIO,A,H,45.9%,15.6%,38.5%
5,ATLETICO-GO,CEARA,D,D,30.6%,53.2%,16.2%
6,INTERNACIONAL,FLAMENGO,A,H,38.2%,29.5%,32.4%
7,FLUMINENSE,AMERICA-MG,H,H,43.3%,35.7%,21.1%
8,BAHIA,CUIABA,D,D,30.6%,39.0%,30.3%
9,SAOPAULO,ATHLETICO-PR,D,H,41.4%,39.1%,19.5%


# Testing our model

Let's create a DataFrame to store all the predictions starting from matchweek 8. Read the Readme of the project for better understanding of this part.

In [10]:
all_predictions = pd.DataFrame(columns = ['Home','Away','result','pred','probH','probD','probA'])

In [11]:
for i in np.arange(8,37):
    t = random_forest_matchweek(i)
    all_predictions = all_predictions.append(t)

0.3
0.3
0.7
0.6
0.4
0.5
0.6
0.2
0.4
0.2
0.3
0.6
0.3
0.3
0.6
0.3
0.5
0.4
0.5
0.3
0.6
0.5
0.6
0.4
0.6
0.6
0.7
0.3
0.7


In [12]:
all_predictions

,Home,Away,result,pred,probH,probD,probA
0,FLUMINENSE,ATHLETICO-PR,A,H,51.9%,14.0%,34.0%
1,BAHIA,AMERICA-MG,A,H,71.4%,11.6%,16.9%
2,CORINTHIANS,SAOPAULO,D,H,56.1%,13.9%,30.0%
3,ATLETICO-MG,ATLETICO-GO,H,H,51.9%,16.0%,32.1%
4,BRAGANTINO,CEARA,D,D,32.4%,67.6%,0.0
...,...,...,...,...,...,...,...
5,BAHIA,GREMIO,H,H,49.0%,18.2%,32.9%
6,ATLETICO-MG,FLUMINENSE,H,H,79.8%,11.7%,8.5%
7,CORINTHIANS,ATHLETICO-PR,H,H,49.6%,18.5%,32.0%
8,BRAGANTINO,AMERICA-MG,D,D,39.7%,41.2%,19.1%


Now, we will make the probs columns numeric and calculate the accuracy of our model.

In [13]:
all_predictions['probH'] = all_predictions['probH'].str.replace("%","")
all_predictions['probD'] = all_predictions['probD'].str.replace("%","")
all_predictions['probA'] = all_predictions['probA'].str.replace("%","")

all_predictions['probH'] = pd.to_numeric(all_predictions['probH'])
all_predictions['probD'] = pd.to_numeric(all_predictions['probD'])
all_predictions['probA'] = pd.to_numeric(all_predictions['probA'])

#### Getting precision

In [14]:
print("Precision when predicting H: {:.2f}".format(calc_precision(all_predictions,'H')))
print("Precision when predicting A: {:.2f}".format(calc_precision(all_predictions,'A')))
print("Precision when predicting D: {:.2f}".format(calc_precision(all_predictions,'D')))

Precision when predicting H: 0.56
Precision when predicting A: 0.41
Precision when predicting D: 0.33


Of course we would be happier with a more accurate predict, but let's keep in mind that match results are not normal distributions and we can get more right predictions with our model than with some random classification. Let's check that out.

In [15]:
print("Percentage of real result H starting from matchweek 8: {:.2f}".format(len(all_predictions[all_predictions['result'] == 'H']) / len(all_predictions)))
print("Percentage of real result A starting from matchweek 8: {:.2f}".format(len(all_predictions[all_predictions['result'] == 'A']) / len(all_predictions)))
print("Percentage of real result D starting from matchweek 8: {:.2f}".format(len(all_predictions[all_predictions['result'] == 'D']) / len(all_predictions)))

Percentage of real result H starting from matchweek 8: 0.46
Percentage of real result A starting from matchweek 8: 0.26
Percentage of real result D starting from matchweek 8: 0.28


As we can see, if we'd say all the matches would be H, A or D, we would get a worse score than with our model.

But we can extract more refined statements from this. Let's consider the precision score based on the probabilities given by our model.

In [16]:
columns=['occurencies', 'precision']
index = np.linspace(35,100,14)
precision_H = pd.DataFrame(columns=columns, index=index)
precision_D = pd.DataFrame(columns=columns, index=index)
precision_A = pd.DataFrame(columns=columns, index=index)
precision_H.index.name = 'prob'
precision_D.index.name = 'prob'
precision_A.index.name = 'prob'

In [17]:
occurencies = []
precision = []
for prob in precision_H.index:
    occurencies.append(calc_precision_with_perc(all_predictions, 'H', prob)[0])
    precision.append(calc_precision_with_perc(all_predictions, 'H', prob)[1])
precision_H['occurencies'] = occurencies
precision_H['precision'] = precision

occurencies = []
precision = []
for prob in precision_D.index:
    occurencies.append(calc_precision_with_perc(all_predictions, 'D', prob)[0])
    precision.append(calc_precision_with_perc(all_predictions, 'D', prob)[1])
precision_D['occurencies'] = occurencies
precision_D['precision'] = precision

occurencies = []
precision = []
for prob in precision_A.index:
    occurencies.append(calc_precision_with_perc(all_predictions, 'A', prob)[0])
    precision.append(calc_precision_with_perc(all_predictions, 'A', prob)[1])
precision_A['occurencies'] = occurencies
precision_A['precision'] = precision

In [18]:
precision_H.style.set_caption('Accuracy when predicting H')

,occurencies,precision
prob,,
35.0,183,0.519126
40.0,133,0.563910
45.0,94,0.585106
50.0,58,0.655172
55.0,34,0.676471
60.0,17,0.647059
65.0,13,0.692308
70.0,9,0.555556
75.0,5,1.000000


That's great. We had a 52% precision with **H** predictions, but we can see that when we get a predicted probability of at least 50% of being **H**, our precision goes up to 63%

In [19]:
precision_D.style.set_caption('Accuracy when predicting D')

,occurencies,precision
prob,,
35.0,127,0.307087
40.0,85,0.329412
45.0,53,0.377358
50.0,28,0.357143
55.0,13,0.384615
60.0,4,0.500000
65.0,3,0.666667
70.0,1,1.000000
75.0,1,1.000000


In [20]:
precision_A.style.set_caption('Accuracy when predicting A')

,occurencies,precision
prob,,
35.0,75,0.373333
40.0,48,0.312500
45.0,30,0.366667
50.0,15,0.400000
55.0,5,0.400000
60.0,4,0.500000
65.0,1,1.000000
70.0,0,0.000000
75.0,0,0.000000


--------

# Predicting real time matches

1. Get the match link from https://ge.globo.com/futebol/brasileirao-serie-a/
2. Put all the links together in a list separated by comma (like the links list below)
3. Note: even if you are predicting only 1 match, it should be inside a list
4. call predict(links)

Note that sometimes the website opens with a new interface they are testing with some users. If this happens, the function *predict* will crash, but you can just run the function again until it goes to the old interface. I promise that when they give me the new interface, I'll update the scraping bot very fast. :)

In [21]:
links = [
    'https://ge.globo.com/rj/futebol/brasileirao-serie-a/jogo/06-12-2021/flamengo-santos.ghtml',
    'https://ge.globo.com/rs/futebol/brasileirao-serie-a/jogo/06-12-2021/internacional-atletico-go.ghtml',
    'https://ge.globo.com/mt/futebol/brasileirao-serie-a/jogo/06-12-2021/cuiaba-fortaleza.ghtml',
    'https://ge.globo.com/sp/futebol/brasileirao-serie-a/jogo/06-12-2021/sao-paulo-juventude.ghtml'
]

In [22]:
predict(links)

,HTEAM,ATEAM,pred,probH,probA,probD
360,FLAMENGO,SANTOS,H,0.616014,0.166975,0.217011
361,INTERNACIONAL,ATLETICO-GO,H,0.463902,0.283217,0.252881
362,CUIABA,FORTALEZA,H,0.420446,0.388523,0.191031
363,SAOPAULO,JUVENTUDE,D,0.358029,0.254963,0.387008


---------